In [ ]:
# Your Python code here
!pip install transformers accelerate peft bitsandbytes datasets torch

In [2]:
from google.colab import drive
import os
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
coldstart_path = input("Enter the full path where the coldstart data should be saved (e.g., mydrive/coldstart_data): ")
os.environ["COLDSTART_FOLDER_PATH"] = coldstart_path
coldstart_file_path = os.getenv("COLDSTART_FOLDER_PATH") #  Coldstart file path ffor initial train data

In [ ]:
from datasets import load_dataset, DatasetDict, load_dataset, Dataset

# Define file paths based on the coldstart_file_path
train_file = os.path.join(coldstart_file_path, "train.jsonl")
valid_file = os.path.join(coldstart_file_path, "valid.jsonl")
test_file = os.path.join(coldstart_file_path, "test.jsonl")

# checking the dataset format
with open(train_file, "r", encoding="utf-8") as file:
    for i in range(5):  # Read first 5 lines
        print(file.readline().strip())  # Print each line

In [ ]:
# checking the dataset format
with open(valid_file, "r", encoding="utf-8") as file:
    for i in range(5):  # Read first 5 lines
        print(file.readline().strip())  # Print each line

In [ ]:
# checking the dataset format
with open(test_file, "r", encoding="utf-8") as file:
    for i in range(5):  # Read first 5 lines
        print(file.readline().strip())  # Print each line

In [8]:
import json

# Function to load JSONL file into a Hugging Face dataset
def load_jsonl_as_dataset(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        data = [json.loads(line.strip()) for line in file]  # Load all lines efficiently
    return Dataset.from_list(data)


In [20]:
dataset_train = load_jsonl_as_dataset(train_file)
dataset_valid = load_jsonl_as_dataset(valid_file)
dataset_test = load_jsonl_as_dataset(test_file)

In [21]:
# Print dataset structure
print(dataset_train, "\n", dataset_valid, "\n", dataset_test)

Dataset({
    features: ['text'],
    num_rows: 382
}) 
 Dataset({
    features: ['text'],
    num_rows: 14
}) 
 Dataset({
    features: ['text'],
    num_rows: 26
})


In [ ]:
type(dataset_train)

In [23]:
print(dataset_valid["text"][0])

<|im_start|>user
Hi, how are you?<|im_end|>
<|im_start|>assistant
Hello! I’m doing well, thanks for asking. How can I help you today?<|im_end|>


In [24]:
import re

def extract_qa_from_message(message):
    """
    Extracts the user question and assistant answer from a given message.
    """
    # Regex pattern to match <|im_start|>user and <|im_start|>assistant segments
    user_pattern = r"<\|im_start\|>user\s*(.*?)<\|im_end\|>"
    assistant_pattern = r"<\|im_start\|>assistant\s*(.*?)<\|im_end\|>"

    # Extract question and answer using regex
    user_match = re.search(user_pattern, message, re.DOTALL)
    assistant_match = re.search(assistant_pattern, message, re.DOTALL)

    # Get extracted text or None
    question = user_match.group(1).strip() if user_match else None
    answer = assistant_match.group(1).strip() if assistant_match else None

    return question, answer

def create_qa_dataset(messages):
    """
    Processes a list of messages and creates a dataset with 'question' and 'answer' columns.
    """
    questions, answers = [], []

    for message in messages:
        question, answer = extract_qa_from_message(message)
        if question and answer:  # Ensure both exist
            questions.append(question)
            answers.append(answer)

    # Convert to Hugging Face Dataset format
    dataset = Dataset.from_dict({"question": questions, "answer": answers})
    return dataset


In [25]:
# function for creating dataset with question and answer columns

def process_messages_to_qa_dataset(messages):
    """
    Extracts questions and answers from a list of messages and returns a structured dataset.
    """
    questions, answers = [], []

    for message in messages:
        # Regex patterns to extract user and assistant messages
        user_match = re.search(r"<\|im_start\|>user\s*(.*?)<\|im_end\|>", message, re.DOTALL)
        assistant_match = re.search(r"<\|im_start\|>assistant\s*(.*?)<\|im_end\|>", message, re.DOTALL)

        # Extracted content
        question = user_match.group(1).strip() if user_match else None
        answer = assistant_match.group(1).strip() if assistant_match else None

        if question and answer:  # Ensure valid Q&A pairs
            questions.append(question)
            answers.append(answer)

    # Convert extracted data to a Hugging Face Dataset
    return Dataset.from_dict({"question": questions, "answer": answers})


In [26]:
qa_dataset_train = process_messages_to_qa_dataset(dataset_train["text"])
qa_dataset_valid = process_messages_to_qa_dataset(dataset_valid["text"])
qa_dataset_test = process_messages_to_qa_dataset(dataset_test["text"])

In [27]:
print(qa_dataset_train.column_names)
print(qa_dataset_train[1])

['question', 'answer']
{'question': 'If a car travels 60 miles in 2 hours, what is its average speed?', 'answer': '<think>Average speed is distance divided by time: 60 miles ÷ 2 hours = 30 miles per hour.</think>\n<answer>30 miles per hour.</answer>'}


In [ ]:
import pandas as pd

# Define the save path
save_path = input("Enter the full path where the modified coldstart csv should be saved: ")

# Ensure the directory exists
os.makedirs(save_path, exist_ok=True)

# Convert to DataFrame and save
pd.DataFrame(qa_dataset_train).to_csv(save_path + "coldstart_dataset_train.csv", index=False)
pd.DataFrame(qa_dataset_valid).to_csv(save_path + "coldstart_dataset_valid.csv", index=False)
pd.DataFrame(qa_dataset_test).to_csv(save_path + "coldstart_dataset_test.csv", index=False)

print(f"Datasets saved to {save_path}")
